In [1]:
import numpy as np
import scipy


def method_CI(A):
    def calc_CI(A_trimmed):
        N = A_trimmed.shape[0]
        lam_max = np.linalg.eigvals(A_trimmed).max()
        return (lam_max - N) / (N-1)
     
    def find_outlier(A):
        CI_min = 1e10
        i_lowest = None
        j_lowest = None
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                CI = calc_CI(np.delete(np.delete(A, i, axis=0), j, axis=1))
                if CI <= CI_min:
                    CI_min = CI
                    i_lowest = i
                    j_lowest = j
        return i_lowest, j_lowest
    
    i_lowest, j_lowest = find_outlier(A)
    print(f"method CI: Outlier is D{(i_lowest+1, j_lowest+1)} (one-based). value = {A[i_lowest, j_lowest]}")


def method_corr(A):
    def find_outlier(A):
        N = A.shape[0]
        
        corr = []
        for i in range(A.shape[0]):
            row_x = A[i]
            std_x = np.std(row_x)
            m_corr = 0
            for i_y in range(A.shape[0]):
                if i_y == i: continue
                row_y = A[i_y]
                std_y = np.std(row_y)
                m_corr += 1.0 / (N - 1) * np.sum((row_x - row_x.mean()) * (row_y - row_y.mean())) / std_x / std_y 
            m_corr /= N - 1
            corr.append(m_corr)
        i_lowest = np.argmin(corr)
        
        corr = []
        for j in range(A.shape[1]):
            col_x = A[:, j]
            std_x = np.std(col_x)
            m_corr = 0
            for j_y in range(A.shape[1]):
                if j_y == j: continue
                col_y = A[:, j_y]
                std_y = np.std(col_y)
                m_corr += 1.0 / (N - 1) * np.sum((col_x - col_x.mean()) * (col_y - col_y.mean())) / std_x / std_y 
            m_corr /= N - 1
            corr.append(m_corr)
        j_lowest = np.argmin(corr)
            
        return i_lowest, j_lowest
    
    i_lowest, j_lowest = find_outlier(A)
    print(f"method corr: Outlier is D{(i_lowest+1, j_lowest+1)} (one-based). value = {A[i_lowest, j_lowest]}")


def method_chi_sq(A):
    N = A.shape[0]
    Delta = np.empty((N, N))
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            t_ij = A[i,:].sum() * A[:,j].sum() / A.sum()
            Delta[i,j] = (A[i,j] - t_ij)**2 / t_ij

    delta_std = Delta.flatten().std()
    delta_mean = Delta.mean()
    print("method chi sq: Outliers are | ", end="")
    for i in range(N):
        for j in range(N):
            if Delta[i, j] > delta_mean + delta_std * 1.645 / np.sqrt(N):
                print(f"D{i+1, j+1} value = {A[i,j]} | ", end="")
    print()


def method_transitive(A):
    N = A.shape[0]
    S = np.empty((N, N))
    i_max = -1
    j_max = -1
    S_ij_max = float("-inf")
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            S_ij = np.sum(A[i,j] * A[j, :] / A[i, :] + A[i,:] / A[i, j] / A[j, :] - 2)
            if S_ij > S_ij_max:
                S_ij_max = S_ij
                i_max = i
                j_max = j
    print(f"method Transitive: Outlier is D{(i_max+1, j_max+1)} (one-based). value = {A[i_max, j_max]}")


def method_M_OutFlow(A):
    N = A.shape[0]
    Phi_pos = np.empty(N)
    Phi_neg = np.empty(N)
    for i in range(A.shape[0]):
        Phi_pos[i] = (A[i, :] > 1).sum()
        Phi_neg[i] = (A[:, i] > 1).sum()

    gamma_max = float("-inf")
    max_val = float("-inf")
    i_max = -1
    j_max = -1

    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i, j] <= 1: continue
                
            val = max(Phi_pos[j] - Phi_pos[i], Phi_neg[i] - Phi_neg[j])
            gamma_ = np.log(A[i, j] - np.log(A[i, :] * A[:, j]))
            mask = np.ones(gamma_.shape[0])
            mask[i] = 0.0
            mask[j] = 0.0
            gamma = 1.0 / (N - 2.0) * np.sum(gamma_ * mask)
            
            
            if val > max_val or (val == max_val and gamma >= gamma_max):
                i_max = i
                j_max = j
                max_val = val
                gamma_max = gamma

    print(f"method M_OutFlow: Outlier is D{(i_max+1, j_max+1)} (one-based). value = {A[i_max, j_max]}")

In [2]:
A = np.array([
    [1.0, 1.0, 0.5, 4.0, 3.0, 0.333, 2.0],
    [1.0, 1.0, 0.5, 4.0, 3.0, 0.333, 2.0],
    [2.0, 7.0, 1.0, 8.0, 6.0, 0.5, 4.0],
    [0.25, 0.25, 0.125, 1.0, 1.0, 7.0, 0.5],
    [0.333, 0.333, 0.167, 1.0, 1.0, 0.111, 1.0],
    [3.0, 3.0, 2.0, 0.142, 9.0, 1.0, 6.0],
    [0.5, 0.5, 0.25, 2.0, 1.0, 0.167, 1.0],
])

In [3]:
method_CI(A)

method CI: Outlier is D(6, 4) (one-based). value = 0.142


In [4]:
method_corr(A)

method corr: Outlier is D(4, 6) (one-based). value = 7.0


In [5]:
method_chi_sq(A)

method chi sq: Outliers are | D(4, 6) value = 7.0 | D(6, 4) value = 0.142 | 


In [6]:
method_transitive(A)

method Transitive: Outlier is D(6, 4) (one-based). value = 0.142


In [7]:
method_M_OutFlow(A)

method M_OutFlow: Outlier is D(4, 6) (one-based). value = 7.0
